In [ ]:
!pip install --upgrade Pillow
import fastbook
fastbook.setup_book()

In [ ]:
!pip install kaggle
# go to kaggle and install your unique api key - move this into the location

In [ ]:
!kaggle datasets download -d kaiska/apparel-dataset
# move dataset into the correct location